In [1]:
pip install requests beautifulsoup4 pandas lxml


In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

def scrape_analyticsvidhya_courses():
    base_url = "https://www.analyticsvidhya.com/courses/"
    all_courses = []

    # Scrape 11 pages
    for page in range(1, 12):
        print(f"Scraping page {page}...")
        url = f"{base_url}?page={page}" if page > 1 else base_url

        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }

        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')

            # Find all course cards
            cards = soup.find_all('div', class_='col-12 col-md-6 col-lg-4')

            for card in cards:
                try:
                    course_data = {}

                    # Title
                    title_elem = card.find('h2', class_='card-text')
                    course_data['title'] = title_elem.get_text(strip=True) if title_elem else 'N/A'

                    # Duration and Lessons (from card-body-top)
                    duration_lessons = card.find('p', class_='fs-14 mb-3 fw-light text-dark-tertiary')
                    if duration_lessons:
                        text = duration_lessons.get_text(strip=True)
                        # Extract duration (Hours) and lessons
                        hours_match = re.search(r'(\d+(?:\.\d+)?)\s*Hour', text)
                        lessons_match = re.search(r'(\d+)\s*Lesson', text)
                        course_data['duration_hours'] = float(hours_match.group(1)) if hours_match else 0
                        course_data['lessons'] = int(lessons_match.group(1)) if lessons_match else 0
                    else:
                        course_data['duration_hours'] = 0
                        course_data['lessons'] = 0

                    # Rating (star rating)
                    rating_elem = card.find('div', class_='float-end fs-16 text-white fw-light d-flex align-items-center')
                    if rating_elem:
                        rating_text = rating_elem.get_text(strip=True)
                        rating_match = re.search(r'(\d+\.?\d*)', rating_text)
                        course_data['rating'] = float(rating_match.group(1)) if rating_match else 0
                    else:
                        course_data['rating'] = 0

                    # Enrollments (from button with SVG user icon)
                    enroll_btn = card.find('button', class_=lambda x: x and 'no-pointer' in x)
                    if enroll_btn:
                        enroll_text = enroll_btn.get_text(strip=True)
                        enroll_match = re.search(r'(\d+)', enroll_text)
                        course_data['enrollments'] = int(enroll_match.group(1)) if enroll_match else 0
                    else:
                        course_data['enrollments'] = 0

                    # Price (check enroll button in footer)
                    enroll_btn_footer = card.find('button', class_='btn-dark-primary')
                    if enroll_btn_footer and 'Free' in enroll_btn_footer.get_text():
                        course_data['price'] = 'Free'
                    else:
                        course_data['price'] = 'Paid'  # Assuming others are paid

                    # Platform
                    course_data['platform'] = 'Analytics Vidhya'

                    # Level (not clearly visible, set as N/A or extract if available)
                    course_data['level'] = 'N/A'

                    # Add page info
                    course_data['page'] = page

                    all_courses.append(course_data)

                except Exception as e:
                    print(f"Error parsing card on page {page}: {e}")
                    continue

            print(f"Found {len(cards)} courses on page {page}")
            time.sleep(1)  # Be respectful to server

        except Exception as e:
            print(f"Error scraping page {page}: {e}")
            continue

    # Save to CSV
    df = pd.DataFrame(all_courses)
    df.to_csv('analyticsvidhya_courses.csv', index=False)
    print(f"\nScraping complete! Saved {len(all_courses)} courses to 'analyticsvidhya_courses.csv'")

    return df

# Run the scraper
if __name__ == "__main__":
    courses_df = scrape_analyticsvidhya_courses()
    print("\nSample data:")
    print(courses_df.head())


Scraping page 1...
Found 544 courses on page 1
Scraping page 2...
Found 544 courses on page 2
Scraping page 3...
Found 544 courses on page 3
Scraping page 4...
Found 544 courses on page 4
Scraping page 5...
Found 544 courses on page 5
Scraping page 6...
Found 544 courses on page 6
Scraping page 7...
Found 544 courses on page 7
Scraping page 8...
Found 544 courses on page 8
Scraping page 9...
Found 544 courses on page 9
Scraping page 10...
Found 544 courses on page 10
Scraping page 11...
Found 544 courses on page 11

Scraping complete! Saved 5984 courses to 'analyticsvidhya_courses.csv'

Sample data:
                                               title  duration_hours  lessons  \
0                 Claude Code:  The Coding Assistant             0.0        0   
1       Claude 4.5: Smarter, Faster  & More Human AI             0.0        3   
2  NotebookLM Essentials to Pro: The Complete Pra...             0.0        0   
3                        ChatGPT  as  Your Assistant             1.0 

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

def scrape_analyticsvidhya_complete():
    """
    Complete Analytics Vidhya course scraper with:
    ✅ Multiple domains/categories
    ✅ Lessons & Duration extraction (FIXED)
    ✅ Rating, Enrollments, Price
    ✅ Enhanced error handling
    ✅ Category detection from title
    """

    # Category and Domain mappings (from Analytics Vidhya filters)
    CATEGORIES = {
        'ai_agents': 'AI/ML', 'prompt_engineering': 'AI/ML', 'nlp': 'AI/ML',
        'deep_learning': 'AI/ML', 'genai': 'Generative AI', 'llm': 'Generative AI',
        'data_analyst': 'Data Analytics', 'business_analytics': 'Business Analytics',
        'vibe_coding': 'Software Development', 'python': 'Programming',
        'strands_agent': 'AI Agents', 'agents': 'AI Agents'
    }

    DOMAINS = {
        'AI/ML': 'Technology', 'Generative AI': 'Technology', 'AI Agents': 'Technology',
        'Data Analytics': 'Analytics', 'Business Analytics': 'Business',
        'Software Development': 'Technology', 'Programming': 'Technology'
    }

    base_url = "https://www.analyticsvidhya.com/courses/"
    all_courses = []

    print("🚀 Starting Analytics Vidhya Complete Scraper...")
    print("📚 Scraping 11 pages with categories, lessons, duration...")

    # Scrape 11 pages
    for page in range(1, 12):
        print(f"\n📄 Scraping page {page}/11...")
        url = f"{base_url}?page={page}" if page > 1 else base_url

        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate',
            'Connection': 'keep-alive',
        }

        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')

            # Find all course cards
            cards = soup.find_all('div', class_='col-12 col-md-6 col-lg-4')
            print(f"   Found {len(cards)} course cards")

            for i, card in enumerate(cards):
                try:
                    course_data = {}

                    # 1. TITLE
                    title_elem = card.find('h2', class_='card-text')
                    course_data['title'] = title_elem.get_text(strip=True) if title_elem else 'N/A'

                    # 2. DURATION & LESSONS (ENHANCED EXTRACTION)
                    duration_lessons_elem = card.find('p', class_='fs-14 mb-3 fw-light text-dark-tertiary')
                    course_data['duration_hours'] = 0
                    course_data['lessons'] = 0

                    if duration_lessons_elem:
                        text = duration_lessons_elem.get_text(strip=True)
                        print(f"   Debug - Duration text: '{text}'")

                        # Multiple regex patterns for duration
                        duration_patterns = [
                            r'(\d+(?:\.\d+)?)\s*(?:Hour|hr|Hours)',  # 5 Hour, 5.5 Hours, 5 hr
                            r'(\d+(?:,\d+)?)\s*(?:Hour|hr|Hours)',  # 1,000 Hours
                            r'(\d+)\s*h(?:ou)?r?',                   # 5h, 5hr
                        ]

                        lessons_patterns = [
                            r'(\d+)\s*(?:Lesson|Lessons|LESSON|lessons)',
                            r'(\d+)\s*lesson',
                        ]

                        # Extract duration
                        for pattern in duration_patterns:
                            match = re.search(pattern, text, re.IGNORECASE)
                            if match:
                                course_data['duration_hours'] = float(match.group(1).replace(',', ''))
                                break

                        # Extract lessons
                        for pattern in lessons_patterns:
                            match = re.search(pattern, text, re.IGNORECASE)
                            if match:
                                course_data['lessons'] = int(match.group(1))
                                break

                    # 3. RATING (IMPROVED)
                    rating_elem = card.find('div', class_='float-end fs-16 text-white fw-light d-flex align-items-center')
                    course_data['rating'] = 0
                    if rating_elem:
                        rating_text = rating_elem.get_text(separator=' ', strip=True)
                        rating_match = re.search(r'(\d+\.?\d*)', rating_text)
                        if rating_match:
                            course_data['rating'] = float(rating_match.group(1))

                    # 4. ENROLLMENTS (ENHANCED)
                    enroll_btn = card.find('button', class_=lambda x: x and 'no-pointer' in x)
                    course_data['enrollments'] = 0
                    if enroll_btn:
                        enroll_text = enroll_btn.get_text(strip=True)
                        # Handle both 1,234 and 1234 formats
                        enroll_match = re.search(r'(\d+(?:,\d+)?(?:k|K| thousand|Thousand)?)', enroll_text)
                        if enroll_match:
                            num_str = enroll_match.group(1).replace(',', '').lower()
                            if 'k' in num_str:
                                course_data['enrollments'] = int(float(num_str.replace('k', '')) * 1000)
                            else:
                                course_data['enrollments'] = int(num_str)

                    # 5. PRICE
                    enroll_btn_footer = card.find('button', class_='btn-dark-primary')
                    if enroll_btn_footer and 'Free' in enroll_btn_footer.get_text(strip=True):
                        course_data['price'] = 'Free'
                    else:
                        course_data['price'] = 'Paid'

                    # 6. PLATFORM & OTHER FIELDS
                    course_data['platform'] = 'Analytics Vidhya'
                    course_data['level'] = 'N/A'
                    course_data['page'] = page
                    course_data['url'] = ''  # Could extract course URL if needed

                    # 7. CATEGORY & DOMAIN DETECTION (INTELLIGENT)
                    title_lower = course_data['title'].lower()
                    detected_category = 'General'
                    detected_domain = 'Technology'

                    # Check categories in title
                    for cat_key, cat_value in CATEGORIES.items():
                        if any(keyword in title_lower for keyword in [cat_key, cat_value.lower().replace(' ', '_')]):
                            detected_category = cat_value
                            detected_domain = DOMAINS.get(cat_value, 'Technology')
                            break

                    course_data['category'] = detected_category
                    course_data['domain'] = detected_domain

                    all_courses.append(course_data)
                    print(f"   ✓ {i+1}/{len(cards)}: {course_data['title'][:50]}... ({course_data['lessons']} lessons, {course_data['duration_hours']}h)")

                except Exception as card_error:
                    print(f"   ❌ Error parsing card {i+1}: {card_error}")
                    continue

            time.sleep(2)  # Be respectful

        except Exception as page_error:
            print(f"   ❌ Page {page} error: {page_error}")
            continue

    # 8. SAVE ENHANCED DATASET
    if all_courses:
        df = pd.DataFrame(all_courses)

        # Reorder columns for better readability
        column_order = [
            'title', 'platform', 'category', 'domain', 'level', 'price',
            'duration_hours', 'lessons', 'rating', 'enrollments', 'page'
        ]
        df = df.reindex(columns=[col for col in column_order if col in df.columns] +
                       [col for col in df.columns if col not in column_order])

        filename = 'analyticsvidhya_courses_complete.csv'
        df.to_csv(filename, index=False)

        print(f"\n🎉 SCRAPING COMPLETE!")
        print(f"📊 Total courses: {len(df)}")
        print(f"⏱️  Total duration: {df['duration_hours'].sum():.1f} hours")
        print(f"📖 Total lessons: {df['lessons'].sum():,} lessons")
        print(f"⭐ Saved to: {filename}")

        # Category & Domain summary
        print("\n📈 CATEGORY BREAKDOWN:")
        print(df['category'].value_counts().to_string())
        print("\n🌐 DOMAIN BREAKDOWN:")
        print(df['domain'].value_counts().to_string())

        # Sample with all fields
        print("\n📋 SAMPLE DATA (Top 5):")
        sample_cols = ['title', 'category', 'domain', 'duration_hours', 'lessons', 'rating', 'enrollments', 'price']
        print(df[sample_cols].head().to_string(index=False))

        return df
    else:
        print("❌ No courses found!")
        return pd.DataFrame()

# BONUS: Category-specific scraper
def scrape_specific_category(category_filter):
    """Scrape only specific category using filter URL"""
    filters = {
        'ai_ml': 'https://www.analyticsvidhya.com/courses/?topics=ai_agents',
        'data_analytics': 'https://www.analyticsvidhya.com/courses/?topics=business_analytics',
        'genai': 'https://www.analyticsvidhya.com/courses/?topics=genai'
    }

    if category_filter in filters:
        print(f"🎯 Scraping ONLY {category_filter.upper()} category...")
        # Reuse main logic with filtered URL
        pass

# RUN THE SCAPER
if __name__ == "__main__":
    print("=" * 80)
    print("🔥 ANALYTICS VIDHYA COMPLETE COURSE SCRAPER")
    print("✅ Categories + Domains + Lessons + Duration + All Fields")
    print("=" * 80)

    courses_df = scrape_analyticsvidhya_complete()

    if not courses_df.empty:
        print(f"\n✅ SUCCESS! {len(courses_df)} courses scraped with complete metadata!")
        print("\n💾 Check 'analyticsvidhya_courses_complete.csv'")
    else:
        print("\n❌ No data scraped. Check internet connection or site changes.")


Streaming output truncated to the last 5000 lines.
   Debug - Duration text: '1 Hour1 Lesson'
   ✓ 242/544: GenAI for Quantitative  Finance & Control Implemen... (1 lessons, 1.0h)
   Debug - Duration text: '1 Hour1 Lesson'
   ✓ 243/544: Creating Problem-Solving Agents with  GenAI for Ac... (1 lessons, 1.0h)
   Debug - Duration text: '4 Hours3 Lessons'
   ✓ 244/544: Generative AI - A Way of Life... (3 lessons, 4.0h)
   Debug - Duration text: '30 Minutes5 Lessons'
   ✓ 245/544: K-Nearest Neighbors (KNN) Algorithm in Python and ... (5 lessons, 0h)
   Debug - Duration text: '1 Hour30 Minutes9 Lessons'
   ✓ 246/544: Fundamentals  of Regression Analysis... (9 lessons, 1.0h)
   Debug - Duration text: '1 Hour9 Lessons'
   ✓ 247/544: Pandas  for Data Analysis in Python... (9 lessons, 1.0h)
   Debug - Duration text: '1 Hour2 Lessons'
   ✓ 248/544: Building Database Agents... (2 lessons, 1.0h)
   Debug - Duration text: '45 Minutes1 Lesson'
   ✓ 249/544: Building a Customized Newsletter AI Agent..